In [7]:
state_dict = torch.load('/home/jupyter-iec_iot13_toanlm/src/src_data4/alzheimer_model_data4.pth')
print(state_dict.keys())

odict_keys(['module.model.conv1.weight', 'module.model.bn1.weight', 'module.model.bn1.bias', 'module.model.bn1.running_mean', 'module.model.bn1.running_var', 'module.model.bn1.num_batches_tracked', 'module.model.layer1.0.conv1.weight', 'module.model.layer1.0.bn1.weight', 'module.model.layer1.0.bn1.bias', 'module.model.layer1.0.bn1.running_mean', 'module.model.layer1.0.bn1.running_var', 'module.model.layer1.0.bn1.num_batches_tracked', 'module.model.layer1.0.conv2.weight', 'module.model.layer1.0.bn2.weight', 'module.model.layer1.0.bn2.bias', 'module.model.layer1.0.bn2.running_mean', 'module.model.layer1.0.bn2.running_var', 'module.model.layer1.0.bn2.num_batches_tracked', 'module.model.layer1.1.conv1.weight', 'module.model.layer1.1.bn1.weight', 'module.model.layer1.1.bn1.bias', 'module.model.layer1.1.bn1.running_mean', 'module.model.layer1.1.bn1.running_var', 'module.model.layer1.1.bn1.num_batches_tracked', 'module.model.layer1.1.conv2.weight', 'module.model.layer1.1.bn2.weight', 'module.

In [29]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

class AlzheimerNet(nn.Module):
    def __init__(self, num_classes=3):  # 3 classes: AD, CN, MCI
        super(AlzheimerNet, self).__init__()
        self.model = resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Load the trained model
model = AlzheimerNet(num_classes=3)
# Load the saved state_dict
state_dict = torch.load('/home/jupyter-iec_iot13_toanlm/src/src_data4/alzheimer_model_data4.pth')

new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace('module.', '')  # remove `module.` prefix
    new_state_dict[name] = v

# Load the modified state_dict
model = AlzheimerNet(num_classes=3)
model.load_state_dict(new_state_dict)
model.eval()  # Set the model to evaluation mode


/opt/tljh/user/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/tljh/user/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlzheimerNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [41]:
from PIL import Image
import os
import sys
from torch.utils.data import Dataset, DataLoader, Subset
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
sys.path.append('/home/jupyter-iec_iot13_toanlm/')
from src.src_data4.dataloadder import MRIDataset
from sklearn.model_selection import train_test_split

# ============================================================
dataset = MRIDataset(root_dir="/home/jupyter-iec_iot13_toanlm/data/data4/data_2D", transform=transform)

train_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.5, stratify=dataset.labels)

train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=4)

# ==============================================================
# data_dir='/home/jupyter-iec_iot13_toanlm/data/data6'
# batch_size=32 
# train_size=0.8
# data_transforms = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
    
#     # Load toàn bộ dataset
# image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms)

# # Chia thành train và test
# train_size = int(train_size * len(image_datasets))
# test_size = len(image_datasets) - train_size
# train_dataset, test_dataset = random_split(image_datasets, [train_size, test_size])

# # Tạo DataLoader cho train và test
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print(len(test_loader))


478


In [42]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(0)
model.to(device)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# Đánh giá model trên bộ dữ liệu test
model.eval()  # Chuyển mô hình sang chế độ đánh giá

all_predictions = []
all_image_paths = []

with torch.no_grad():  # Không cần tính toán gradient khi test
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_predictions.extend(predicted.cpu().numpy())  # Lưu kết quả dự đoán
        all_image_paths.extend(img_paths)  # Lưu đường dẫn của các ảnh

# Hiển thị kết quả dự đoán
# for img_path, prediction in zip(all_image_paths, all_predictions):
#     print(f"Image: {img_path}, Predicted class: {prediction}")


In [43]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# all_predictions 
# all_image_paths 

accuracy = accuracy_score(all_image_paths , all_predictions)
f1 = f1_score(all_image_paths , all_predictions, average='weighted')
precision = precision_score(all_image_paths , all_predictions, average='weighted')
recall = recall_score(all_image_paths , all_predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Accuracy: 0.9543
F1 Score: 0.9543
Precision: 0.9545
Recall: 0.9543
